This notebook will preprocess all of the data

In [60]:
# imports

import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from PIL import Image, ImageFilter

In [78]:
# create label dict
label_dic = {'N': 0, 'O': 1, 'R': 2}
path = 'DATASET/'

# index 0-2 is train, 3-5 is test
data = []
paths = ["DATASET/TRAIN/N", "DATASET/TRAIN/O", "DATASET/TRAIN/R", "DATASET/TEST/N", "DATASET/TEST/O", "DATASET/TEST/R"]

for i, path in enumerate(paths):
    print(path)
    data.append(os.listdir(path))
    print(data[i][:5])
    print(len(data[i]))

# print(data[0][:10])

DATASET/TRAIN/N
['R_1.jpg', 'R_10.jpg', 'R_100.jpg', 'R_1000.jpg', 'R_1001.jpg']
2847
DATASET/TRAIN/O
['O_1.jpg', 'O_10.jpg', 'O_100.jpg', 'O_1000.jpg', 'O_10000.jpg']
12565
DATASET/TRAIN/R
['R_2848.jpg', 'R_2849.jpg', 'R_2850.jpg', 'R_2851.jpg', 'R_2852.jpg']
7152
DATASET/TEST/N
['00000000.jpg', '00000001.jpg', '00000002.jpg', '00000003.jpg', '00000004.jpg']
397
DATASET/TEST/O
['O_12568.jpg', 'O_12569.jpg', 'O_12570.jpg', 'O_12571.jpg', 'O_12572.jpg']
1401
DATASET/TEST/R
['R_10000.jpg', 'R_10001.jpg', 'R_10002.jpg', 'R_10003.jpg', 'R_10004.jpg']
1112


In [79]:
# preprocess each image into a matrix of 500 x 500 pixels
train_data = []
train_labels = []

test_data = []
test_labels = []

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

for i, category in enumerate(data):
    print(category[0])
    for j, image in enumerate(category):
        img = Image.open(paths[i] + "/" + image).convert('L')
        width, height = img.size
        img = add_margin(img, 0, 500 - width, 500 - height, 0, 255)
        print(img.mode)
        # Calculating Edges using the passed laplican Kernel
        final = img.filter(ImageFilter.Kernel((3, 3), (-1, -1, -1, -1, 8,
                                                -1, -1, -1, -1), 1, 0))
        img.show()
        final.show()
        category[j] = final
        
        if(j == 3):
            break
    break

print("Done with image sizing")

R_1.jpg
L
L
L
L
Done with image sizing


In [ ]:
# run edge detection on the images


In [ ]:
# split into validation sets

In [ ]:
# train model

In [ ]:
# test data on testing set